# Рабочий вариант курсовой работы

## Настройка ноутбука

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Загрузка данных

* **data_test** - test.csv
* **data_train** - train.csv
* **data_all** - совмещенные test и train (для анализа распределения, расчета производных значений, определения классов)
* **data** - данные train которые будем очищать

In [16]:
data_test = pd.read_csv('test.csv')
data_train = pd.read_csv('train.csv')
data_all = pd.concat([data_test, data_train.drop(['Price'], axis=1)])
data = data_train.copy()

## Анализ данных по столбцам на аномалии

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


1. Поле Healthcare_1 уберем, т.к. в нем слишком много пропущенных значений (боле 47%)

In [18]:
data = data.drop(['Healthcare_1'], axis=1)

2. Поле Id проверим на уникальность по data_all

In [21]:
data_all['Id'].nunique() == data_all.shape[0]

True

**Вывод**: все квартиры разные, все в порядке

3. Заполнять пропущенные значения в поле LifeSquare будем средним значением LifeSquare для каждой группы квартир по количеству комнат на всех данных (data_all). Но перед этим необходимо почистить данные в полях Rooms и LifeSquare.

In [22]:
columns = ['Rooms', 'LifeSquare']
data[columns].describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,10000.0,1.890500,0.839512,0.000000,1.000000,2.00000,2.000000,19.000000
LifeSquare,7887.0,37.199645,86.241209,0.370619,22.769832,32.78126,45.128803,7480.592129


Необходимо проверить:

+ квартиры с кол-ом комнат равном нулю, это либо ошибка, либо квартиры студии
+ квартиры с большим количеством комнат
+ аномально минимальные значения жилой площади
+ аномально макисмальные значения жилой площади

In [24]:
data[data['Rooms'] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,0,1,B,219281.918007


Для квартиры 1397 попробуем ориентироваться на цену и год постройки, т.к. сомнительно выглядят данные по полям Rooms, Square, LifeSquare, а так же Floor > HouseFloor

In [27]:
data[data['HouseYear'] == 2016].groupby(by=['Rooms'])['Price'].agg(['mean', 'std', 'max', 'min'])

,mean,std,max,min
Rooms,,,,
0.0,268394.744389,NaN,268394.744389,268394.744389
1.0,150770.731336,78666.573500,594565.299770,65949.315881
2.0,202262.123085,75163.322661,437452.971467,71291.546163
3.0,269861.595968,111375.854360,577122.841727,113751.304561
4.0,313697.530698,87767.195168,401035.038716,237969.144323


Видно, что квартира 1397 скорее всего 3-х комнатная.

Попробуем определить значения площадей.

In [30]:
data[(data['HouseYear'] == 2016) & (data['Rooms'] == 3)][['Square', 'LifeSquare']].describe()

,Square,LifeSquare
count,52.000000,39.000000
mean,83.576946,257.321648
std,17.961169,1187.472132
min,4.390331,1.626501
25%,79.306995,49.101988
50%,83.246030,76.857588
75%,88.963092,88.382169
max,124.601250,7480.592129


Значение Square для квартиры 1397 можно взять равным 83.576946